<a href="https://colab.research.google.com/github/1MperaDOR0/Gerador-de-Tamplates-para-WebSites/blob/main/Gerador_de_Tamplates_para_WebSites_e_Duvidas_em_Programacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Instalando o SDK do Google
!pip install -q -U google-generativeai
!pip install gdown

# Importando bibliotecas necessárias de Python
import google.generativeai as genai
import numpy as np
import pandas as pd
import os
import gdown

# Fazendo a linkagem da API do Google AI Studio
GOOGLE_API_KEY = "SUA_API_KEY"  # Substitua pela sua API Key
genai.configure(api_key=GOOGLE_API_KEY)

# Definindo as configurações de geração de conteúdo
generation_config = {
    'candidate_count': 1,
    'temperature': 0.5,
}
# Definindo as configurações de segurança
safety_settings = {
    'Harassment': 'BLOCK_NONE',
    'Hate': 'BLOCK_NONE',
    'Sexual': 'BLOCK_NONE',
    'Dangerous': 'BLOCK_NONE',
}
# Inicializando o modelo
model = genai.GenerativeModel(
    model_name='gemini-1.0-pro',
    generation_config=generation_config,
    safety_settings=safety_settings
)

# Criando o histórico do chat
chat = model.start_chat(history=[])

# Melhorando a visualização
# Código disponível em https://ai.google.dev/tutorials/python_quickstart#import_packages
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Função para ler o conteúdo de um arquivo do Google Drive
def ler_arquivo(id_arquivo):
  url = f'https://drive.google.com/uc?id={id_arquivo}&export=download'
  output = 'temp_file'
  gdown.download(url, output, quiet=True)
  with open(output, 'r') as arquivo:
    conteudo = arquivo.read()
  return conteudo

# Carregando os documentos e gerando embeddings
documents = []
for i in range(1, 4):  # Loop para os documentos 1 a 3
  html_id = f'1OvAWXVIrbCx4iyGorLneMR9wYUj_vc4x'  # ID do arquivo HTML
  css_id = f'1vlfe0CpnialN4vx7NTP1mitBso7ncMFD'  # ID do arquivo CSS

  documento = {
      'Arquivo HTML': ler_arquivo(html_id),
      'Arquivo CSS': ler_arquivo(css_id),
      'embedding': None
  }
  documents.append(documento)

df = pd.DataFrame(documents)

# Gerando embeddings para cada documento
embedding_model = "models/embedding-001"
for index, row in df.iterrows():
  conteudo = row['Arquivo HTML'] + row['Arquivo CSS']  # Combinando HTML e CSS
  embedding = genai.embed_content(model=embedding_model, content=conteudo, task_type='RETRIEVAL_QUERY')['embedding']
  df.at[index, 'embedding'] = embedding

# Função para buscar o documento mais relevante
def gerar_e_buscar_consulta(prompt, df, embedding_model):
  embedding_da_consulta = genai.embed_content(model=embedding_model, content=prompt, task_type='RETRIEVAL_QUERY')['embedding']
  produtos_escalares = np.dot(np.stack(df['embedding'].values), embedding_da_consulta)
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]

modo_programacao = True

# Histórico da conversa
conversation_history = []

# Loop principal do chatbot
prompt = input('\nDigite seu prompt (digite "finalizar" para parar):\n')
while prompt.strip().lower() != "finalizar":

    if prompt.strip().lower() == "/programacao":
        modo_programacao = not modo_programacao
        print(f"Modo programação: {'Ativado' if modo_programacao else 'Desativado'}")
        prompt = input('\nDigite seu prompt (digite "finalizar" para parar):\n')
        continue

    documento_relevante = gerar_e_buscar_consulta(prompt, df, embedding_model)
    conteudo_html = documento_relevante['Arquivo HTML']
    conteudo_css = documento_relevante['Arquivo CSS']

    mensagem_completa = f"""
    {prompt}
    ## Contexto HTML:
    {conteudo_html}
    ## Contexto CSS:
    {conteudo_css}
    """

    if modo_programacao:
        mensagem_completa =  "Responda apenas sobre a área de TI (Tecnologia da Informação), porém com foco no desenvolvimento de tamplates para WebSites (mas só quando requisitado), usando linguagem simples e exemplos. Se a pergunta for sobre outro assunto, faça algo parecido com esse exemplo: ""Desculpe, mas como programado, só posso responder perguntas relacionadas a área de TI. 🐔 não é um tema da área. Você pode fazer perguntas relacionadas a programação, ciência da computação, análise de dados, criação de tamplates para WebSites, entre outras coisas da área de TI.\n" + mensagem_completa

    # Incluindo o histórico na mensagem
    message_with_history = "\n".join(conversation_history) + "\n" + mensagem_completa

    # Enviando a mensagem com o histórico
    response = chat.send_message(message_with_history)

    # Adicionando a mensagem atual e a resposta ao histórico
    conversation_history.append(mensagem_completa)
    conversation_history.append(response.text)

    print('\nResposta ao prompt:\n', response.text, '\n')
    prompt = input('Digite seu prompt (digite "finalizar" para parar):\n')


Digite seu prompt (digite "finalizar" para parar):
Crie um tamplate para um site de restaurante.

Resposta ao prompt:
 **Template para Site de Restaurante**

**Contexto HTML:**

```html
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link rel="stylesheet" type="text/css" href="style.css"/>
    <title>Nome do Restaurante</title>
</head>
<body>

    <header>
        <h1>Nome do Restaurante</h1>

        <nav>
            <a href="#">Página Inicial</a>
            <a href="#">Cardápio</a>
            <a href="#">Sobre Nós</a>
            <a href="#">Contato</a>
        </nav>
    </header>

    <main>
        <section id="home">
            <h2>Página Inicial</h2>

            <p>Bem-vindo ao nosso restaurante! Aqui você encontra uma experiência gastronômica inesquecível.</p>
            <p>Oferecemos um cardápio variado, com opções para todos os gostos. Venha nos visitar e experimente nossos delicio